# End of Week 1 Exercise: Technical Q&A Tool

This notebook demonstrates your familiarity with both the OpenAI API and Ollama by building a tool that answers technical questions with clear explanations. You can use this tool throughout the course to get help with technical concepts.

## Instructions
- The notebook is fully documented for clarity and reproducibility.
- You can ask any technical question in the input cell below.
- The tool will respond using either OpenAI or Ollama, depending on your selection.
- Make sure you have API keys and/or Ollama installed and running locally.


In [9]:
import os
from dotenv import load_dotenv
import requests
import json
from typing import Generator

In [10]:
MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [11]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENROUTER_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-or-"):
    print("An API key was found, but it doesn't start sk-or-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [12]:
# here is the question; type over this to ask something new
question = """
How does transfer learning improve the performance of deep neural networks in domains with limited labeled data?
"""


In [13]:
# Get gpt-4o-mini to answer, with streaming
# The prompt is concise and focused on technical explanation

def stream_gpt4o_answer(question: str, api_key: str) -> Generator[str, None, None]:
    url = "https://openrouter.ai/api/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    data = {
        "model": MODEL_GPT,
        "messages": [
            {"role": "system", "content": "You are an expert technical explainer. Respond with a clear, concise, and accurate explanation to the user's technical question."},
            {"role": "user", "content": question}
        ],
        "stream": True
    }
    with requests.post(url, headers=headers, data=json.dumps(data), stream=True) as resp:
        for line in resp.iter_lines():
            if line:
                try:
                    # OpenRouter streams lines starting with 'data: '
                    if line.startswith(b'data: '):
                        payload = line[6:]
                        if payload.strip() == b"[DONE]":
                            break
                        chunk = json.loads(payload)
                        content = chunk.get('choices', [{}])[0].get('delta', {}).get('content')
                        if content:
                            yield content
                except Exception as e:
                    print(f"Error parsing stream: {e}")
answer_stream = stream_gpt4o_answer(question, api_key)
print(''.join(answer_stream))


Transfer learning improves the performance of deep neural networks in domains with limited labeled data by leveraging knowledge gained from pretrained models on larger, related datasets. Here’s how it works:

1. **Pretraining**: A deep learning model is first trained on a large dataset (often on a general task). During this phase, the model learns to capture general features and representations from the data.

2. **Feature Extraction**: The pretrained model can serve as a feature extractor. When applied to a new task with limited labeled data, the model can utilize the learned features rather than starting from scratch. This allows it to make better predictions even when the new dataset is small.

3. **Fine-tuning**: The pretrained model can be further fine-tuned on the new dataset. This involves training the model for a few more epochs on the limited data, allowing it to adjust its weights while still retaining the knowledge from the original training. Fine-tuning helps the model adap

In [14]:
# Get Llama 3.2 to answer
# The prompt is concise and focused on technical explanation

def stream_llama_answer(question: str, api_key: str) -> Generator[str, None, None]:
    url = "http://localhost:11434/api/generate"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    # Ollama expects a single prompt string, not messages
    prompt = f"You are an expert technical explainer. Respond with a clear, concise, and accurate explanation to the user's technical question.\n{question}"
    data = {
        "model": MODEL_LLAMA,
        "prompt": prompt,
        "stream": True
    }
    with requests.post(url, headers=headers, data=json.dumps(data), stream=True) as resp:
        for line in resp.iter_lines():
            if line:
                try:
                    chunk = json.loads(line)
                    content = chunk.get('response', '')
                    if content:
                        yield content
                    if chunk.get('done', False):
                        break
                except Exception as e:
                    print(f"Error parsing stream: {e}")
llama_answer_stream = stream_llama_answer(question, "ollama")
print(''.join(llama_answer_stream))


Transfer learning is a machine learning technique that allows pre-trained models to be fine-tuned on new, but possibly limited datasets. By leveraging knowledge acquired from one task (source domain) and adapting it to another task (target domain), transfer learning can significantly improve the performance of deep neural networks in domains with limited labeled data.

Here's how transfer learning works:

1. **Pre-training**: A pre-trained model is trained on a large, diverse dataset in the source domain. This process allows the model to learn general features and representations that are applicable across multiple tasks.
2. **Fine-tuning**: The pre-trained model is then fine-tuned on a smaller, target dataset in the target domain. During this phase, the model's weights are updated based on the new data, allowing it to adapt to the specific task at hand.

Transfer learning improves performance in domains with limited labeled data by:

* **Reducing overfitting**: Fine-tuning reduces the

In [15]:
class LLMConversation:
    def __init__(self, api_key: str, top_k: int = 20, top_p: float = 0.9):
        self.api_key = api_key
        self.history = []
        self.models = {
            "gpt": MODEL_GPT,
            "llama": MODEL_LLAMA
        }
        self.top_k = top_k
        self.top_p = top_p

    def stream_answer(self, model: str, context: str) -> str:
        output = ""
        if model == "gpt":
            url = "https://openrouter.ai/api/v1/chat/completions"
            headers = {
                "Authorization": f"Bearer {self.api_key}",
                "Content-Type": "application/json"
            }
            messages = [{"role": "system", "content": "You are a helpful technical explainer."}]
            messages += [{"role": "user", "content": context}]
            data = {
                "model": self.models[model],
                "messages": messages,
                "stream": True,
                "top_p": self.top_p,
                "max_tokens": 150
            }
            print(f"{model}: ")
            with requests.post(url, headers=headers, data=json.dumps(data), stream=True) as resp:
                for line in resp.iter_lines():
                    if line and line.startswith(b'data: '):
                        payload = line[6:]
                        if payload.strip() == b"[DONE]":
                            break
                        chunk = json.loads(payload)
                        content = chunk.get('choices', [{}])[0].get('delta', {}).get('content')
                        if content:
                            print(content, end='', flush=True)
                            output += content
            print()
        elif model == "llama":
            url = "http://localhost:11434/api/generate"
            headers = {
                "Authorization": f"Bearer {self.api_key}",
                "Content-Type": "application/json"
            }
            prompt = f"You are a helpful technical explainer.\n{context}"
            data = {
                "model": self.models[model],
                "prompt": prompt,
                "stream": True,
                "top_k": self.top_k,
                "top_p": self.top_p,
                "max_tokens": 150
            }
            print(f"{model}: ")
            with requests.post(url, headers=headers, data=json.dumps(data), stream=True) as resp:
                for line in resp.iter_lines():
                    if line:
                        try:
                            chunk = json.loads(line)
                            content = chunk.get('response', '')
                            if content:
                                print(content, end='', flush=True)
                                output += content
                            if chunk.get('done', False):
                                break
                        except Exception as e:
                            print(f"Error parsing stream: {e}")
            print()
        return output

    def converse(self, topic: str, turns: int = 4, start_with: str = "gpt"):
        if not self.history:
            self.history = [topic]
        current = start_with
        for i in range(turns):
            context = "\n".join(self.history)
            print(f"\n--- {current.upper()} turn ---")
            answer = self.stream_answer(current, context)
            self.history.append(answer)
            current = "llama" if current == "gpt" else "gpt"

In [16]:
llm_chat = LLMConversation(api_key=api_key)
llm_chat.converse("Effect of AI and agents on software engineering")


--- GPT turn ---
gpt: 
The impact of AI and intelligent agents on software engineering is profound and multifaceted, influencing various stages of the software development lifecycle (SDLC) and transforming traditional practices. Here are some of the key effects:

### 1. **Automation of Repetitive Tasks**
   - **Code Generation**: AI tools like GitHub Copilot and OpenAI Codex can assist developers by generating code snippets or even entire functions based on natural language prompts. This reduces the time spent on boilerplate code and speeds up development.
   - **Testing and Debugging**: AI can automate testing processes, identifying bugs and vulnerabilities more efficiently than manual testing. Tools like Selenium can be enhanced with AI to improve test coverage and accuracy.

### 2. **Enhanced Software

--- LLAMA turn ---
llama: 
### 2. **Enhanced Software Development**

   - **Feature Engineering and Design**: AI agents can assist in feature engineering by suggesting new features b